In [ ]:
# 9/17/24

# I will try to create a model to predict if users will click on an ad or not.
# I will try to make a decision tree, knn, and svm model to predict this.


In [1]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import csv
import json
import os
from pandas import DataFrame, Series

import seaborn as sns

from itertools import combinations


from sklearn import datasets
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LinearRegression
lr = LinearRegression()

from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor()

from sklearn.neighbors import KNeighborsRegressor
knr = KNeighborsRegressor()

from sklearn.svm import SVR
svr = SVR()


from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import VotingRegressor


from scipy.stats import ttest_rel


In [2]:
dtr = DecisionTreeRegressor()
knr = KNeighborsRegressor()
svr = SVR()


In [86]:
# Read the ad click data
df = pd.read_csv('ad_click_dataset.csv')

# Display the first 5 rows of the data
df.head()

# So I can see the click is the target column. I will try to predict this column.
# I will also need to add default values, possibly, for all the NaN values.

# And eventually, I can drop the id and full_name columns. Before any model making. 



,id,full_name,age,gender,device_type,ad_position,browsing_history,time_of_day,click
0,670,User670,22.0,NaN,Desktop,Top,Shopping,Afternoon,1
1,3044,User3044,NaN,Male,Desktop,Top,NaN,NaN,1
2,5912,User5912,41.0,Non-Binary,NaN,Side,Education,Night,1
3,5418,User5418,34.0,Male,NaN,NaN,Entertainment,Evening,1
4,9452,User9452,39.0,Non-Binary,NaN,NaN,Social Media,Morning,0


In [87]:
# Check column types.
df.dtypes


id                    int64
full_name            object
age                 float64
gender               object
device_type          object
ad_position          object
browsing_history     object
time_of_day          object
click                 int64
dtype: object

In [88]:
# Check data shape.
df.shape


(10000, 9)

In [89]:
# There are many missing values. 
# To address this, I will fill in the missing values for numeric columns with the mean of the column.
# And for the object columns, I will fill in the missing values with the mode of the column.

# I will also drop the id and full_name columns.

print(df['age'].isna().sum())

df['age'] = df['age'].fillna(df['age'].mean())

print(df['age'].isna().sum())

df['gender'] = df['gender'].fillna(df['gender'].mode()[0])
df['device_type'] = df['device_type'].fillna(df['device_type'].mode()[0])
df['ad_position'] = df['ad_position'].fillna(df['ad_position'].mode()[0])
df['browsing_history'] = df['browsing_history'].fillna(df['browsing_history'].mode()[0])
df['time_of_day'] = df['time_of_day'].fillna(df['time_of_day'].mode()[0])

if 'id' in df.columns and 'full_name' in df.columns:
    df = df.drop(columns=['id', 'full_name'])

df.head()


4766
0


,age,gender,device_type,ad_position,browsing_history,time_of_day,click
0,22.000000,Female,Desktop,Top,Shopping,Afternoon,1
1,40.197363,Male,Desktop,Top,Entertainment,Morning,1
2,41.000000,Non-Binary,Desktop,Side,Education,Night,1
3,34.000000,Male,Desktop,Bottom,Entertainment,Evening,1
4,39.000000,Non-Binary,Desktop,Bottom,Social Media,Morning,0


In [90]:
df['age'].unique()

df.iloc[325:331]


,age,gender,device_type,ad_position,browsing_history,time_of_day,click
325,40.197363,Male,Tablet,Bottom,Entertainment,Night,0
326,18.000000,Female,Desktop,Bottom,Entertainment,Afternoon,0
327,40.000000,Non-Binary,Mobile,Side,Entertainment,Morning,1
328,55.000000,Non-Binary,Desktop,Side,Entertainment,Evening,1
329,40.197363,Female,Tablet,Side,Shopping,Morning,0
330,40.197363,Female,Mobile,Top,Entertainment,Morning,1


In [91]:
# Next, I will make the age column into a categorical column.
# I will make the age column into 3 equal categories.

# Bin the age column into 3 equal-sized bins
df['age'] = pd.cut(df['age'], bins=3)

# Convert the binned column to string type
df['age'] = df['age'].astype(str)

# Show the resulting dataframe
df.head()


,age,gender,device_type,ad_position,browsing_history,time_of_day,click
0,"(17.954, 33.333]",Female,Desktop,Top,Shopping,Afternoon,1
1,"(33.333, 48.667]",Male,Desktop,Top,Entertainment,Morning,1
2,"(33.333, 48.667]",Non-Binary,Desktop,Side,Education,Night,1
3,"(33.333, 48.667]",Male,Desktop,Bottom,Entertainment,Evening,1
4,"(33.333, 48.667]",Non-Binary,Desktop,Bottom,Social Media,Morning,0


In [92]:
df.dtypes


age                 object
gender              object
device_type         object
ad_position         object
browsing_history    object
time_of_day         object
click                int64
dtype: object

In [94]:
# Okay so I got the data cleaned up.
# But I guess I need to do one hot encoding on categorical data to get it to work 
# with sci-kit learn models.

# I will use the OneHotEncoder to do this.

# Define the column transformer, applying OneHotEncoder to the first 6 columns
ct = ColumnTransformer([("encoder", OneHotEncoder(sparse_output=False), list(range(6)))],
remainder="passthrough")  # Pass through the numeric target column unchanged

# Apply the transformer to the DataFrame
new_data = ct.fit_transform(df)

type(ct)

ct.get_feature_names_out()

# Convert the transformed data into a new DataFrame with the new feature names and the original index
df_encoded = pd.DataFrame(new_data, columns=ct.get_feature_names_out(), index=df.index)

# Output the encoded DataFrame
df_encoded.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 22 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   encoder__age_(17.954, 33.333]            10000 non-null  float64
 1   encoder__age_(33.333, 48.667]            10000 non-null  float64
 2   encoder__age_(48.667, 64.0]              10000 non-null  float64
 3   encoder__gender_Female                   10000 non-null  float64
 4   encoder__gender_Male                     10000 non-null  float64
 5   encoder__gender_Non-Binary               10000 non-null  float64
 6   encoder__device_type_Desktop             10000 non-null  float64
 7   encoder__device_type_Mobile              10000 non-null  float64
 8   encoder__device_type_Tablet              10000 non-null  float64
 9   encoder__ad_position_Bottom              10000 non-null  float64
 10  encoder__ad_position_Side                10000 

In [95]:
df_encoded.head()


,"encoder__age_(17.954, 33.333]","encoder__age_(33.333, 48.667]","encoder__age_(48.667, 64.0]",encoder__gender_Female,encoder__gender_Male,encoder__gender_Non-Binary,encoder__device_type_Desktop,encoder__device_type_Mobile,encoder__device_type_Tablet,encoder__ad_position_Bottom,...,encoder__browsing_history_Education,encoder__browsing_history_Entertainment,encoder__browsing_history_News,encoder__browsing_history_Shopping,encoder__browsing_history_Social Media,encoder__time_of_day_Afternoon,encoder__time_of_day_Evening,encoder__time_of_day_Morning,encoder__time_of_day_Night,remainder__click
0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
1,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
2,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0
3,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
4,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [98]:
# Now the data is ready for classification models.

# I will make a decision tree model first.

parameters = [{"min_samples_leaf":[5,10,50,100,200]}]

tuned_dtr = GridSearchCV(dtr,
parameters, scoring="neg_root_mean_squared_error",
cv=5)

# Notice the column names changed in the df_encoded. 'click' is now 'remainder__click'.
scores = cross_validate(tuned_dtr, df_encoded.drop(columns="remainder__click"), 
df_encoded.remainder__click, cv=10, scoring="neg_root_mean_squared_error")


scores["test_score"]

dtr_rmse = rmse = 0-scores["test_score"]
rmse.mean()
dtr_rmse
print("dtr_rmse = " + str(dtr_rmse.mean()))


dtr_rmse = 0.46944429535305626


In [99]:
# Now I will make the knn model.

parameters = [{"n_neighbors":[3,5,7,9,11]}]

tuned_knn = GridSearchCV(knr,
parameters, scoring="neg_root_mean_squared_error",
cv=5)

scores = cross_validate(tuned_knn, df_encoded.drop(columns="remainder__click"), 
df_encoded.remainder__click, cv=10, scoring="neg_root_mean_squared_error")


scores["test_score"]

knn_rmse = rmse = 0-scores["test_score"]
rmse.mean()
knn_rmse
print("knn_rmse = " + str(knn_rmse.mean()))



knn_rmse = 0.47353472804992613


In [100]:
# Now I will make the svm model.

scores = cross_validate(svr, df_encoded.drop(columns="remainder__click"), 
df_encoded.remainder__click, cv=10, scoring="neg_root_mean_squared_error")


scores["test_score"]

svm_rmse = rmse = 0-scores["test_score"]
rmse.mean()
svm_rmse
print("svm_rmse = " + str(svm_rmse.mean()))


svm_rmse = 0.49067018296314746
